# Lab 3: SQL Aggregation Queries

### [name]

## Instruction
This lab is due at 11:59 pm on Mar. 20. You need to finish all questions of both In-class Assginment and Homework. Write you name in the above cell. Name your ipynb file as **StudentName_lab3.ipynb** and submit it on Blackboard. 

PLEASE ENSURE THAT THE SCRIPT CAN CORRECTLY EXECUTE!!! 

In [39]:
#Let's connect to a database **lab3.db** using:
%load_ext sql
#%reload_ext sql
%sql sqlite:///lab3.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


We will continue to use the following three tables. 

1. Employee(<u>eid</u>:integer, ename:string, age:integer, salary:real)

2. Works(<u>eid</u>:integer, <u>did</u>:integer, pct_time:integer)

3. Department(<u>did</u>:integer, dname:string, budge:real, managerid:integer)

The keys are underlined in each relation. Relation **Employee** stores employee information such as unique identifier **eid**, employee name **ename**, **age** and **salary**. Relation **Department** stores the department unique identifier **did**, department name **dname**, the department **budget** and **managerid** which is the **eid** of the employee who is managing the department. The **managerid** value must always be found in the **eid** field of a record of the Employee relation. The Works relation tracks which
employee works in which department, and what **percentage** of the time s/he allocates to that department. Note that, an employee can work in several departments.


1. Employee

| eid     | ename         | age | salary |  
|---------|---------------|-----|--------|
| 101     | Andrew Fuller | 40  | 50000  |
| 102     | Justin Peacock| 25  | 15000  |
| 103     | Nancy Buchanan| 35  | 30000  |
| 104     | Robert King   | 30  | 20000  |

2. Works:

| eid     | did  | pct_time | 
|---------|------|----------|
| 101     | 1001 | 100      |
| 102     | 1001 | 20       |
| 102     | 1002 | 50       |
| 102     | 1003 | 10       |
| 103     | 1003 | 90       |
| 103     | 1001 | 10       |
| 104     | 1001 | 20       |
| 104     | 1002 | 70       |

3. Department:

| did     | dname    | budget  | managerid |
|---------|----------|---------|-----------|
| 1001    | IT       | 500000  | 101       |
| 1002    | Computing| 750000  | 110       |
| 1003    | Finance  | 350000  | 103       |

In [2]:
%sql Select * from Employee

 * sqlite:///lab3.db
Done.


eid,ename,age,salary
101,Andrew Fuller,40,50000
102,Justin Peacock,25,15000
103,Nancy Buchanan,35,30000
104,Robert King,30,20000


## In-class Assignment

### Exercise 1  [6 points]
Find the average salary over all employees.

In [40]:
%%sql
Select Avg(E.salary)
From Employee E
Where E.salary

 * sqlite:///lab3.db
Done.


Avg(E.salary)
28750.0


### Exercise 2  [6 points]
Find the average age of employees for each department where every employee is 30 years old or
younger. Please list the corresponding department id as well.

In [14]:
%%sql
Select Avg(E.age), D.did
From Employee E, Department D, Works W
Where (D.did = W.did AND
       W.eid = E.eid)
Group By D.did
Having E.age <= 30

 * sqlite:///lab3.db
Done.


Avg(E.age),did
27.5,1002
30.0,1003


### Exercise 3  [6 points]
Find for each manager (listed in the output by eid) the average salary of employees working for
that manager. 

In [37]:
%%sql
Select Avg(E.Salary), D.managerid
From Employee E, Department D, Works W
Where (E.eid = W.eid AND
       D.did = W.did)
Group By D.managerid

 * sqlite:///lab3.db
Done.


Avg(E.Salary),managerid
28750.0,101
22500.0,103
17500.0,110


### Exercise 4  [6 points]
Find the maximum salary among employees 30 years old or yonger for each department with at least 2 employees of any age. Please list the corresponding department id as well.

In [46]:
%%sql
Select Max(E.salary), D.did
From Works W, Employee E, Department D
Where (E.age <= 30 AND
       E.eid = W.eid AND
       D.did = W.did)
Group By D.did
Having 2 <= (Select Count(*)
           From Employee E1)

 * sqlite:///lab3.db
Done.


Max(E.salary),did
20000,1001
20000,1002
15000,1003


### Exercise 5  [8 points]
Find the name(s) of department(s) who have the highest average employee age.

In [76]:
%%sql
WITH Temp AS (
    Select D.dname, AVG(E.age) AS avgage
    From Department D, Employee E
    Group By D.dname)
Select D.dname, Max(Temp.avgage)
From Temp, Department D

#I figured out another way to implement the Temp table without creating it multiple times as you stated in office hours 
#I did this by using the WITH operator (The code doesnt run with these comments, they need to be deleted.)

 * sqlite:///lab3.db
Done.


dname,Max(Temp.avgage)
IT,32.5


## Homework

### Question 1 [9 points]
Find the number of employees in each department. List the department name and its number of employees.

In [33]:
%%sql
Select D.dname, Count(E.eid)
From Employee E, Works W, Department D
Where (E.eid = W.eid AND
       D.did = W.did)
Group By D.did, D.dname

 * sqlite:///lab3.db
Done.


dname,Count(E.eid)
IT,4
Computing,2
Finance,2


### Question 2 [9 points]
Find the total salary among employees 30 years old or yonger for each department with at least 2 such employees.

In [36]:
%%sql
Select D.dname, Sum(E.salary)
From Employee E, Department D, Works W
Where (E.age >=30 AND
       E.eid = W.eid AND
       D.did = W.did)
Group By D.dname
Having 2 <= Count(E.eid)

 * sqlite:///lab3.db
Done.


dname,Sum(E.salary)
IT,100000
